# AWS DevDay Seoul 2019
## 모두를 위한 컴퓨터 비전 딥러닝 툴킷, GluonCV 따라하기
### Lab 3. Object detection 모델을 AWS DeepLens에 배포하기

#### 랩 순서

1. GluonCV 모델을 DeepLens에 배포할 모델 형태로 준비하기
2. DeepLens 모델 정의하기
3. DeepLens 프로젝트 만들기
4. DeepLens에 배포하고 테스트하기

DeepLens에 배포한 모델을 저장할 S3 버킷이름과 모델 파일 이름을 정합니다.

> 주의: S3 버킷 이름은 꼭 **"deeplens-"**로 시작해야합니다.

In [1]:
S3_BUCKET = 'deeplens-mh-s3-us-east-1'
DL_MODEL_NAME = 'model-yolov3.tar.gz'

In [2]:
!aws s3 mb s3://$S3_BUCKET --region us-east-1

make_bucket: deeplens-mh-s3-us-east-1


### Task 1 - GluonCV 모델을 DeepLens에 배포할 모델 형태로 준비하기
http://mxnet.incubator.apache.org/versions/master/tutorials/gluon/save_load_params.html

In [3]:
import mxnet as mx
import gluoncv as gcv

In [4]:
net_name = 'yolo3_darknet53_custom'
model_param_fname = 'yolo3_darknet53_voc_0000_0.0000.params'
classes = ['Bird', 'Boat', 'Car', 'Person']

ctx = mx.gpu()

In [5]:
net = gcv.model_zoo.get_model(net_name, classes=classes, ctx=ctx)
net.load_parameters(model_param_fname, ctx=ctx)

net.hybridize()

x=mx.ndarray.zeros([1, 3, 512, 512], ctx=ctx)
y=net(x)

mx.nd.waitall()
net.export('deeplens-yolov3')

모델의 심볼 파일(.json)과 파라메터 파일(.params)가 잘 생성되었는지 확인하고, tar.gz 형태로 압축한 후 위에서 지정한 S3 bucket으로 업로드합니다.

In [6]:
!ls -l deeplens-yolov3*

-rw-rw-r-- 1 ec2-user ec2-user 246794208 Sep 12 21:07 deeplens-yolov3-0000.params
-rw-rw-r-- 1 ec2-user ec2-user    217006 Sep 12 21:07 deeplens-yolov3-symbol.json
-rw-rw-r-- 1 ec2-user ec2-user 228907234 Sep 12 01:10 deeplens-yolov3.zip


In [7]:
!tar cvfz ./$DL_MODEL_NAME ./deeplens-yolov3-*
!aws s3 cp ./$DL_MODEL_NAME s3://$S3_BUCKET/models/

./deeplens-yolov3-0000.params
./deeplens-yolov3-symbol.json
upload: ./model-yolov3.tar.gz to s3://deeplens-mh-s3-us-east-1/models/model-yolov3.tar.gz


### Task 2 - DeepLens 모델 정의하기

1. DeepLens 콜솔로 이동 후, 'Resources > Models' 선택
2. 'Import model' 버튼 클릭

![](./images/devday-deeplens-lab-01.png)

3. 'Import source'를 'Externally trained model'로 변경
4. Model settings에 값들을 입력
- Model artifact path: s3://__S3_BUCKET__/models/model-yolov3.tar.gz
- Model name: model 이름 (예, object-detection)
- Model framework: MXNet 선택

![](./images/devday-deeplens-lab-02.png)

### Task 2 - DeepLens 프로젝트 만들기

1. DeepLens 콜솔로 이동 후, 'Resources > Projects' 선택
2. 'Create new project' 버튼 클릭

![](./images/devday-deeplens-lab-03.png)

3. Project type은 'Use a project template' 선택
4. Project templates은 'Object detection' 선택 후, 'Next' 클릭

![](./images/devday-deeplens-lab-04.png)

5. Project name, description 입력 후, 'Create' 버튼 클릭

![](./images/devday-deeplens-lab-05.png)

6. 목록에서 생성된 프로젝트 클릭

![](./images/devday-deeplens-lab-06.png)

7. 모델 및 Lambda 함수 수정을 위해, 'Edit' 버튼 클릭

![](./images/devday-deeplens-lab-07.png)

8. 기본 모델 삭제를 위해서, Model의 'Remove' 버튼 클릭

![](./images/devday-deeplens-lab-08.png)

9. GluonCV로 학습시킨 모델을 추가하기 위해서, 'Add model' 클릭

![](./images/devday-deeplens-lab-09.png)

10. 모델 목록에서, 이전 단계에서 정의한 모델 선택 후 'Add model' 버튼 클ㄹ릭

![](./images/devday-deeplens-lab-10.png)

DeepLens에 연결된 모니터을 통해서 결과를 확인하기 위해서는 DeepLens에 로그인해서 아래 명령을 수행합니다.

```
mplayer -demuxer lavf /opt/awscam/out/ch1_out.h264
```